# Custom scorer

In [2]:
import pandas as pd

In [3]:
# Let's create a dummy "wine" dataset of 1000 wines, 10 features, and 3 classes (0=bad, 1=medium, 2=good wine)
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=1000, n_features=10, n_classes=3, n_clusters_per_class=1, weights=[0.6, 0.3, 0.1], random_state=0
)
print("X.shape = ", X.shape)
pd.Series(y).value_counts()

X.shape =  (1000, 10)


0    599
1    299
2    102
dtype: int64

-----
❓ Our objective is to train a model which **maximizes prediction precision for the good wines (y=2) only**.  

We don't want any customers to be dissatisfied!

----

In [68]:
# We split train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [69]:
# We want to gridsearch SVC classifiers (for instance)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [70]:
# Let's CV the "best" SVC for "accurary" first (default perf metrics for SVC)
param_grid = {
    "kernel": ['rbf', 'linear'],
    "C": [0.1, 0.05, 1, 5, 10, 50, 100],
    "gamma": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}

clf = GridSearchCV(SVC(), param_grid, cv=5)
clf = clf.fit(X_train, y_train)
clf.best_estimator_

SVC(C=100, gamma=0.005)

In [71]:
# Print results
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       176
           1       0.88      0.92      0.90        93
           2       0.80      0.65      0.71        31

    accuracy                           0.91       300
   macro avg       0.87      0.84      0.85       300
weighted avg       0.91      0.91      0.91       300



In [72]:
# What if we want to optimize for precision of feature 2 only?

In [73]:
# We need a metric for our multi-class problem. Let's try sklearn "precision_score"
from sklearn.metrics import precision_score

_true = [0,0,1,2,2,1]
_pred = [0,1,1,2,1,0]

precision_score(_true,_pred, average='weighted')

0.611111111111111

☝️ Not good enough, we want to focus **only** on class 2

In [74]:
# Let's make our own custom metrics which gives only the precision of class "2"
# TP2/(TP2+FN2.1+FN2.0)
def my_custom_metric(y_true, y_pred):
    acc_tot = 0
    acc_tp = 0
    
    for (idx, y_p) in enumerate(y_pred):
        if y_p ==2:
            acc_tot +=1
            if y_true[idx] == 2:
                acc_tp +=1
    
    if acc_tot == 0:
        return 0
    else:
        return acc_tp/acc_tot


_true = [0,0,1,2,2,1]
_pred = [0,1,1,2,1,0]

my_custom_metric(_true, _pred)

1.0

In [ ]:
# Let's try to plug that into sklearn (will crash)
GridSearchCV(SVC(), param_grid, cv=5, scoring=my_custom_metric).fit(X_train, y_train)

In [76]:
# We need to transform our "metric" into a "sklearn scorer method"
from sklearn.metrics import make_scorer
my_custom_scorer = make_scorer(my_custom_metric)

In [77]:
# Then we do another GridSearch with this custom scorer

clf = GridSearchCV(SVC(), param_grid, cv=5, scoring=my_custom_scorer)
clf = clf.fit(X_train, y_train)
clf.best_estimator_

SVC(C=5, gamma=0.0005)

In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93       176
           1       0.79      0.87      0.83        93
           2       1.00      0.26      0.41        31

    accuracy                           0.86       300
   macro avg       0.90      0.70      0.72       300
weighted avg       0.87      0.86      0.84       300



✅ We improved our precision for class 2 from 0.8 up to 1, but at the detriment of overall accuracy!